In [ ]:
import pyomo.environ as pe

# -----------------------------
# Data
# -----------------------------
generators = [
    {'country': 'Argentland', 'tech': 'coal', 'cost': 36, 'capacity': 51000},
    {'country': 'Argentland', 'tech': 'windsolar', 'cost': 0, 'capacity': 9000},
    {'country': 'Argentland', 'tech': 'gas', 'cost': 78, 'capacity': 12500},
    {'country': 'Bronzeland', 'tech': 'hydro', 'cost': 4, 'capacity': 2100},
    {'country': 'Bronzeland', 'tech': 'gas', 'cost': 74, 'capacity': 1000},
    {'country': 'Copperland', 'tech': 'hydro', 'cost': 6, 'capacity': 1400}
]

transmission_lines = {
    ('Argentland', 'Bronzeland'): 600,
    ('Bronzeland', 'Copperland'): 50,
    ('Argentland', 'Copperland'): 200
}

demand = {
    'Argentland': 64000,
    'Bronzeland': 1200,
    'Copperland': 800
}

# -----------------------------
# Model
# -----------------------------
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

# Generator set: (country, tech)
gen_keys = [(g['country'], g['tech']) for g in generators]
model.GENS = pe.Set(initialize=gen_keys, dimen=2)

# Line set: (from, to)
line_keys = list(transmission_lines.keys())
model.LINES = pe.Set(initialize=line_keys, dimen=2)

# Parameters
model.gen_cost = pe.Param(model.GENS, initialize={(g['country'], g['tech']): g['cost'] for g in generators})
model.gen_capacity = pe.Param(model.GENS, initialize={(g['country'], g['tech']): g['capacity'] for g in generators})
model.trans_capacity = pe.Param(model.LINES, initialize=transmission_lines)

# Variables
model.GEN = pe.Var(model.GENS, domain=pe.NonNegativeReals)
model.FLOW = pe.Var(model.LINES, domain=pe.Reals)


In [ ]:
num_var_elements = sum(len(v) for v in model.component_objects(pe.Var, active=True))
print("Number of individual variables:", num_var_elements)

In [ ]:
model.cost = pe.Objective(expr=sum(model.gen_cost[g] * model.GEN[g] for g in model.GENS), sense=pe.minimize)

In [ ]:
@model.Constraint(model.GENS)
def generator_limit(m, c, tech):
    return m.GEN[c, tech] <= model.gen_capacity[c, tech]

In [ ]:
@model.Constraint(demand.keys())
def energy_balance(m, country):
    generation = sum(m.GEN[c, tech] for (c, tech) in m.GENS if c == country)
    inflow = sum(m.FLOW[f, t] for (f, t) in m.LINES if t == country)
    outflow = sum(m.FLOW[f, t] for (f, t) in m.LINES if f == country)
    net_import = inflow - outflow
    return generation + net_import == demand[country]

In [ ]:
pe.SolverFactory("appsi_highs").solve(model).write()

In [2]:
import pyomo.environ as pe

# -----------------------------
# Data
# -----------------------------
generators = [
    {'country': 'Argentland', 'tech': 'coal', 'cost': 36, 'capacity': 51000},
    {'country': 'Argentland', 'tech': 'windsolar', 'cost': 0, 'capacity': 9000},
    {'country': 'Argentland', 'tech': 'gas', 'cost': 78, 'capacity': 12500},
    {'country': 'Bronzeland', 'tech': 'hydro', 'cost': 4, 'capacity': 2100},
    {'country': 'Bronzeland', 'tech': 'gas', 'cost': 74, 'capacity': 1000},
    {'country': 'Copperland', 'tech': 'hydro', 'cost': 6, 'capacity': 1400}
]

transmission_lines = {
    ('Argentland', 'Bronzeland'): 600,
    ('Bronzeland', 'Copperland'): 50,
    ('Argentland', 'Copperland'): 200
}

demand = {
    'Argentland': 64000,
    'Bronzeland': 1200,
    'Copperland': 800
}

# -----------------------------
# Model
# -----------------------------
model = pe.ConcreteModel()
model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

# Sets
gen_keys = [(g['country'], g['tech']) for g in generators]
model.GENS = pe.Set(initialize=gen_keys, dimen=2)
model.LINES = pe.Set(initialize=transmission_lines.keys(), dimen=2)
model.COUNTRIES = pe.Set(initialize=demand.keys())

# Parameters
model.gen_cost = pe.Param(model.GENS, initialize={(g['country'], g['tech']): g['cost'] for g in generators})
model.gen_capacity = pe.Param(model.GENS, initialize={(g['country'], g['tech']): g['capacity'] for g in generators})
model.trans_capacity = pe.Param(model.LINES, initialize=transmission_lines)
model.demand = pe.Param(model.COUNTRIES, initialize=demand)

# Variables
model.GEN = pe.Var(model.GENS, domain=pe.NonNegativeReals)
model.FLOW = pe.Var(model.LINES, domain=pe.Reals)  # Flows can be positive or negative

# Objective: minimize total generation cost
model.cost = pe.Objective(expr=sum(model.gen_cost[g] * model.GEN[g] for g in model.GENS), sense=pe.minimize)

# Constraints

# Generator capacity limit
@model.Constraint(model.GENS)
def generator_limit(m, c, tech):
    return m.GEN[c, tech] <= m.gen_capacity[c, tech]

# Transmission capacity limit (flows limited by line capacity)
@model.Constraint(model.LINES)
def flow_limit(m, f, t):
    return (-m.trans_capacity[f, t], m.FLOW[f, t], m.trans_capacity[f, t])

# Energy balance per country: generation + inflow - outflow = demand
@model.Constraint(model.COUNTRIES)
def energy_balance(m, country):
    generation = sum(m.GEN[c, tech] for (c, tech) in m.GENS if c == country)
    inflow = sum(m.FLOW[f, t] for (f, t) in m.LINES if t == country)
    outflow = sum(m.FLOW[f, t] for (f, t) in m.LINES if f == country)
    return generation + inflow - outflow == m.demand[country]

# -----------------------------
# Solve
# -----------------------------
solver = pe.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# -----------------------------
# Print Results
# -----------------------------
print(f"\nTotal operational cost: {pe.value(model.cost):.2f} €")

print("\nGenerator dispatch (MW):")
for (c, tech) in model.GENS:
    print(f"  {c} - {tech}: {pe.value(model.GEN[c, tech]):.2f} MW")

print("\nPower flows on transmission lines (MW):")
for (f, t) in model.LINES:
    print(f"  Flow from {f} to {t}: {pe.value(model.FLOW[f, t]):.2f} MW")


Running HiGHS 1.10.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
LP   has 12 rows; 9 cols; 21 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e+00, 8e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+01, 6e+04]
Presolving model
2 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-12); columns 0(-9); elements 0(-21) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  2.0987000000e+06
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Total operational cost: 2098700.00 €

Generator dispatch (MW):
  Argentland - coal: 51000.00 MW
  Argentland - windsolar: 9000.00 MW
  Argentland - gas: 3200.00 MW
  Bronzeland - hydro: 1850.00 MW
  Bronzeland - gas: 0.00 MW
  Copperland - hydro: 950.00 MW

Power flows on transmission lines (MW):
  Flow from Argentland to Bronzeland: -600.00 MW
  Flow from Bronzeland to Copperland: